# Data Collection - Pokemon and News Sites
Submitted by Kyle Tagle - S11

### Import Libraries

In [72]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

## Pokemon Web Scraping

In [73]:
#load pokemon page
URL = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
pokemonPage = requests.get(URL)
#print(pokemonPage.content)

#setup bs4
soup = BeautifulSoup(pokemonPage.content, 'html.parser')

In [74]:
#find pokemon generations tables

pokemonAll = soup.find(id="mw-content-text")
generationTables = pokemonAll.find_all("table")

In [75]:
# explore table contents
generationTables[1].contents[3]

<tr style="background:#FFF">
<td style="font-family:monospace"> #001
</td>
<td style="font-family:monospace"> #001
</td>
<th> <a href="/wiki/Bulbasaur_(Pok%C3%A9mon)" title="Bulbasaur"><img alt="Bulbasaur" height="68" src="//cdn.bulbagarden.net/upload/2/21/001MS8.png" width="68"/></a>
</th>
<td> <a href="/wiki/Bulbasaur_(Pok%C3%A9mon)" title="Bulbasaur (Pokémon)">Bulbasaur</a>
</td>
<td colspan="1" style="text-align:center; background:#78C850"> <a href="/wiki/Grass_(type)" title="Grass (type)"><span style="color:#FFF">Grass</span></a> </td>
<td align="center" colspan="1" rowspan="1" style="background:#A040A0"> <a href="/wiki/Poison_(type)" title="Poison (type)"><span style="color:#FFFFFF">Poison</span></a>
</td></tr>

In [76]:
# test pokemon extraction 

urlFormat = "https://bulbapedia.bulbagarden.net"

pokemonTest = generationTables[1].contents[3]
for i in range(len(pokemonTest)):
    print(f"Index {i}: {pokemonTest.contents[i]}")
    
dex = pokemonTest.contents[1].text.strip()
ndex = pokemonTest.contents[3].text.strip()
name = pokemonTest.contents[7].text.strip()
type0 = pokemonTest.contents[9].text.strip()
url = urlFormat+pokemonTest.contents[7].find("a")["href"]
#url = pokemonTest.contents[5].attrs["href"]
#print(type(pokemonTest.contents[3]))
#print(dir(pokemonTest.contents[3]))
print(f'Dex: {dex}\nNDex: {ndex}\nName: {name}\nType: {type0}\nURL: {url}')


Index 0: 

Index 1: <td style="font-family:monospace"> #001
</td>
Index 2: 

Index 3: <td style="font-family:monospace"> #001
</td>
Index 4: 

Index 5: <th> <a href="/wiki/Bulbasaur_(Pok%C3%A9mon)" title="Bulbasaur"><img alt="Bulbasaur" height="68" src="//cdn.bulbagarden.net/upload/2/21/001MS8.png" width="68"/></a>
</th>
Index 6: 

Index 7: <td> <a href="/wiki/Bulbasaur_(Pok%C3%A9mon)" title="Bulbasaur (Pokémon)">Bulbasaur</a>
</td>
Index 8: 

Index 9: <td colspan="1" style="text-align:center; background:#78C850"> <a href="/wiki/Grass_(type)" title="Grass (type)"><span style="color:#FFF">Grass</span></a> </td>
Index 10: 

Index 11: <td align="center" colspan="1" rowspan="1" style="background:#A040A0"> <a href="/wiki/Poison_(type)" title="Poison (type)"><span style="color:#FFFFFF">Poison</span></a>
</td>
Dex: #001
NDex: #001
Name: Bulbasaur
Type: Grass
URL: https://bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)


#### Iterate Through Each Generation and Store to JSON

In [77]:
pokemonJSON = []
for i in range(1,9):
    curGeneration = generationTables[i]
    pokemonStart = 3
    dexType = curGeneration.contents[1].contents[1].text.strip()
    for x in range(pokemonStart, len(curGeneration.contents), 2):
        pokemonDetails = curGeneration.contents[x]
        dex = pokemonDetails.contents[1].text.strip()
        ndex = pokemonDetails.contents[3].text.strip()
        name = pokemonDetails.contents[7].text.strip()
        type0 = pokemonDetails.contents[9].text.strip()
        url = urlFormat+pokemonDetails.contents[7].find("a")["href"]
        if len(pokemonDetails.contents) > 10:
            type1=pokemonDetails.contents[11].text.strip()
            print(f'Pokemon {ndex} {name} is a {type0} & {type1} Pokemon')
            pokemonJSON.append({
                dexType: dex,
                "ndex": ndex,
                "name": name,
                "type1": type0,
                "type2": type1,
                "url": url,
            })
        else:
            print(f'Pokemon {ndex} {name} is a {type1} Pokemon')
            pokemonJSON.append({
                dexType: dex,
                "ndex": ndex,
                "name": name,
                "type1": type0,
                "url": url,
            })      

Pokemon #001 Bulbasaur is a Grass & Poison Pokemon
Pokemon #002 Ivysaur is a Grass & Poison Pokemon
Pokemon #003 Venusaur is a Grass & Poison Pokemon
Pokemon #004 Charmander is a Poison Pokemon
Pokemon #005 Charmeleon is a Poison Pokemon
Pokemon #006 Charizard is a Fire & Flying Pokemon
Pokemon #007 Squirtle is a Flying Pokemon
Pokemon #008 Wartortle is a Flying Pokemon
Pokemon #009 Blastoise is a Flying Pokemon
Pokemon #010 Caterpie is a Flying Pokemon
Pokemon #011 Metapod is a Flying Pokemon
Pokemon #012 Butterfree is a Bug & Flying Pokemon
Pokemon #013 Weedle is a Bug & Poison Pokemon
Pokemon #014 Kakuna is a Bug & Poison Pokemon
Pokemon #015 Beedrill is a Bug & Poison Pokemon
Pokemon #016 Pidgey is a Normal & Flying Pokemon
Pokemon #017 Pidgeotto is a Normal & Flying Pokemon
Pokemon #018 Pidgeot is a Normal & Flying Pokemon
Pokemon #019 Rattata is a Flying Pokemon
Pokemon #019 Rattata is a Dark & Normal Pokemon
Pokemon #020 Raticate is a Normal Pokemon
Pokemon #020 Raticate is a Da

In [78]:
pokemonJSON

[{'Kdex': '#001',
  'ndex': '#001',
  'name': 'Bulbasaur',
  'type1': 'Grass',
  'type2': 'Poison',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)'},
 {'Kdex': '#002',
  'ndex': '#002',
  'name': 'Ivysaur',
  'type1': 'Grass',
  'type2': 'Poison',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Ivysaur_(Pok%C3%A9mon)'},
 {'Kdex': '#003',
  'ndex': '#003',
  'name': 'Venusaur',
  'type1': 'Grass',
  'type2': 'Poison',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Venusaur_(Pok%C3%A9mon)'},
 {'Kdex': '#004',
  'ndex': '#004',
  'name': 'Charmander',
  'type1': 'Fire',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Charmander_(Pok%C3%A9mon)'},
 {'Kdex': '#005',
  'ndex': '#005',
  'name': 'Charmeleon',
  'type1': 'Fire',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Charmeleon_(Pok%C3%A9mon)'},
 {'Kdex': '#006',
  'ndex': '#006',
  'name': 'Charizard',
  'type1': 'Fire',
  'type2': 'Flying',
  'url': 'https://bulbapedia.bulbagarden.net/wiki/Chariza

In [79]:
with open('pokemon.json', 'w') as file:
    json.dump(pokemonJSON, file, indent=4)

## News Site Web Scraping

### Import Libraries
Source: https://newsapi.org/docs/client-libraries/python

In [80]:
import sys
!{sys.executable} -m pip install newsapi-python

from newsapi import NewsApiClient

In [81]:
#init newsapi obj
newsapi = NewsApiClient(api_key='a8c6abf8b6774929b4d2965dd64fb5a3')

all_articles = newsapi.get_everything(sources='abc-news',
                                      domains='https://abcnews.go.com/',
                                      from_param='2021-03-11',
                                      to='2021-03-12',
                                      language='en',
                                      sort_by='relevancy')

In [82]:
print(type(all_articles))

<class 'dict'>


In [83]:
all_articles.pop('status')
print(all_articles)

{'totalResults': 402, 'articles': [{'source': {'id': 'abc-news', 'name': 'ABC News'}, 'author': 'VANESA GERA Associated Press', 'title': 'Poland, Hungary file complaint at EU court over budget rule', 'description': 'The Polish and Hungarian governments have filed a complaint with the European Union’s supreme court challenging a new mechanism that links funding with the respect for rule of law', 'url': 'https://abcnews.go.com/International/wireStory/poland-hungary-file-complaint-eu-court-budget-rule-76386587', 'urlToImage': 'null', 'publishedAt': '2021-03-11T13:35:56Z', 'content': "WARSAW, Poland -- The right-wing governments of Poland and Hungary filed a complaint with the European Union's supreme court on Thursday challenging a new mechanism that links funding with the respec… [+2482 chars]"}, {'source': {'id': 'abc-news', 'name': 'ABC News'}, 'author': 'Ben Gittleson', 'title': 'Biden not yet holding a formal news conference raises accountability questions', 'description': "The White

In [84]:
all_articles.pop('totalResults')
print(all_articles)

{'articles': [{'source': {'id': 'abc-news', 'name': 'ABC News'}, 'author': 'VANESA GERA Associated Press', 'title': 'Poland, Hungary file complaint at EU court over budget rule', 'description': 'The Polish and Hungarian governments have filed a complaint with the European Union’s supreme court challenging a new mechanism that links funding with the respect for rule of law', 'url': 'https://abcnews.go.com/International/wireStory/poland-hungary-file-complaint-eu-court-budget-rule-76386587', 'urlToImage': 'null', 'publishedAt': '2021-03-11T13:35:56Z', 'content': "WARSAW, Poland -- The right-wing governments of Poland and Hungary filed a complaint with the European Union's supreme court on Thursday challenging a new mechanism that links funding with the respec… [+2482 chars]"}, {'source': {'id': 'abc-news', 'name': 'ABC News'}, 'author': 'Ben Gittleson', 'title': 'Biden not yet holding a formal news conference raises accountability questions', 'description': "The White House still hasn't s

In [85]:
articles = all_articles.get('articles')
print(type(articles))

<class 'list'>


In [86]:
#remove unneeeded items
for d in articles:
    del d['source']
    del d['description']
    del d['url']
    del d['urlToImage']
    d['date'] = d.pop('publishedAt')
    d['fullArticle'] = d.pop('content')
print(articles)

[{'author': 'VANESA GERA Associated Press', 'title': 'Poland, Hungary file complaint at EU court over budget rule', 'date': '2021-03-11T13:35:56Z', 'fullArticle': "WARSAW, Poland -- The right-wing governments of Poland and Hungary filed a complaint with the European Union's supreme court on Thursday challenging a new mechanism that links funding with the respec… [+2482 chars]"}, {'author': 'Ben Gittleson', 'title': 'Biden not yet holding a formal news conference raises accountability questions', 'date': '2021-03-11T10:34:48Z', 'fullArticle': 'Halfway into his first 100 days, President Joe Biden has yet to hold a formal, solo news conference, raising questions about accountability with the White House under increasing pressure to explain w… [+6235 chars]'}, {'author': 'PAUL WISEMAN AP Economics Writer', 'title': 'US jobless claims fall to 712,000 as pace of layoffs eases', 'date': '2021-03-11T13:50:30Z', 'fullArticle': 'WASHINGTON -- The number of Americans seeking unemployment benefits

#### Cons of the API
Using the API that I used, it is unable to fully capture the full article in the news websites. 

In [87]:
df = pd.DataFrame(articles)
df.head(10)

,author,title,date,fullArticle
0,VANESA GERA Associated Press,"Poland, Hungary file complaint at EU court ove...",2021-03-11T13:35:56Z,"WARSAW, Poland -- The right-wing governments o..."
1,Ben Gittleson,Biden not yet holding a formal news conference...,2021-03-11T10:34:48Z,"Halfway into his first 100 days, President Joe..."
2,PAUL WISEMAN AP Economics Writer,"US jobless claims fall to 712,000 as pace of l...",2021-03-11T13:50:30Z,WASHINGTON -- The number of Americans seeking ...
3,Dr. Tarun Jain,"HIV testing, treatment decreases during COVID-...",2021-03-11T13:14:50Z,The COVID-19 pandemic may be leading to decrea...
4,ADAM BEAM Associated Press,California could get $150B from federal virus ...,2021-03-11T14:20:31Z,"SACRAMENTO, Calif. -- The massive COVID-19 rel..."
5,The Associated Press,Phoenix Zoo euthanizes 22-year-old lion due to...,2021-03-11T14:23:27Z,PHOENIX -- A 22-year-old female African lion h...
6,The Associated Press,Judge grants prosecution request to add third-...,2021-03-11T14:27:28Z,Judge grants prosecution request to add third-...
7,Allie Yang,Why were hundreds of vaccine appointments canc...,2021-03-11T00:07:39Z,"To prove eligibility for a COVID-19 vaccine, m..."
8,MESFIN FEKADU AP Music Writer,Erivo on near-EGOT status: 'I’m just going to ...,2021-03-11T14:46:42Z,NEW YORK -- The Color Purple turned into Gramm...
9,The Associated Press,BMW remained profitable in 2020 with strong se...,2021-03-11T12:27:13Z,"FRANKFURT, Germany -- German luxury automaker ..."


In [88]:
with open('news.json', 'w') as file:
    json.dump(articles, file, indent=4)